In [9]:
# 匯入必要套件
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import HTML, display
import ipywidgets as widgets
from google.colab import files
uploaded = files.upload()
# 1. 載入模型
# 2. 獲取文件名
model_filename = list(uploaded.keys())[0]
print(f"✅ 已上傳模型: {model_filename}")

# 3. 加載模型
model = load_model(model_filename)
print("✅ 模型加載成功！")

# 2. 建立輸入輸出介面
output = widgets.Output()
display(output)

html_code = """
<div style="font-family: Arial;">
  <h4>請用鼠標繪製數字：</h4>
  <canvas id="drawCanvas" width="ˇ320" height="320"
          style="border:2px solid #666; cursor:crosshair; background:white;"></canvas>
  <div style="margin-top:15px;">
    <button onclick="processDrawing()"
            style="padding:6px 15px; background:#4285f4; color:white; border:none; border-radius:4px;">
      預測
    </button>
    <button onclick="clearCanvas()"
            style="padding:6px 15px; margin-left:10px; background:#f1f1f1; border:1px solid #ddd; border-radius:4px;">
      清除重畫
    </button>
  </div>
  <div id="matrixContainer" style="margin-top:20px; display:none;">
    <h4>生成的矩陣：</h4>
    <div id="matrixOutput"
         style="font-family:monospace; background:#f8f8f8; padding:10px; border-radius:4px;"></div>
    <div style="margin-top:15px;">
      <div style="font-weight:bold; margin-bottom:5px;">視覺化預覽：</div>
      <canvas id="matrixPreview" width="160" height="160"
              style="border:1px solid #ccc; background:white;"></canvas>
    </div>
    <div id="predictOutput" style="margin-top:20px; font-weight:bold; color:#d32f2f;"></div>
  </div>
</div>

<script>
const canvas = document.getElementById('drawCanvas');
const ctx = canvas.getContext('2d');
ctx.strokeStyle = 'black';
ctx.lineWidth = 40;
ctx.lineCap = 'round';
ctx.lineJoin = 'round';

let isDrawing = false;
let lastX = 0;
let lastY = 0;

canvas.addEventListener('mousedown', startDrawing);
canvas.addEventListener('mousemove', draw);
canvas.addEventListener('mouseup', stopDrawing);
canvas.addEventListener('mouseout', stopDrawing);

function startDrawing(e) {
  isDrawing = true;
  [lastX, lastY] = [e.offsetX, e.offsetY];
  ctx.beginPath();
  ctx.moveTo(lastX, lastY);
}

function draw(e) {
  if (!isDrawing) return;
  ctx.lineTo(e.offsetX, e.offsetY);
  ctx.stroke();
  [lastX, lastY] = [e.offsetX, e.offsetY];
}

function stopDrawing() {
  isDrawing = false;
}

function clearCanvas() {
  ctx.fillStyle = 'white';
  ctx.fillRect(0, 0, canvas.width, canvas.height);
  document.getElementById('matrixContainer').style.display = 'none';
}

function processDrawing() {
  const tempCanvas = document.createElement('canvas');
  tempCanvas.width = tempCanvas.height = 8;
  const tempCtx = tempCanvas.getContext('2d');
  tempCtx.drawImage(canvas, 0, 0, 8, 8);

  const imgData = tempCtx.getImageData(0, 0, 8, 8);
  const matrix = [];
  let matrixHTML = '[<br>';

  for (let y = 0; y < 8; y++) {
    const row = [];
    for (let x = 0; x < 8; x++) {
      const idx = (y * 8 + x) * 4;
      const avg = (imgData.data[idx] + imgData.data[idx+1] + imgData.data[idx+2]) / 3;
      row.push(avg < 128 ? 1 : 0);
    }
    matrix.push(row);
    matrixHTML += '  [' + row.join(',') + ']' + (y < 7 ? ',' : '') + '<br>';
  }
  matrixHTML += ']';

  const container = document.getElementById('matrixContainer');
  container.style.display = 'block';
  document.getElementById('matrixOutput').innerHTML = matrixHTML;

  const preview = document.getElementById('matrixPreview');
  const previewCtx = preview.getContext('2d');
  const scale = 20;
  previewCtx.fillStyle = 'white';
  previewCtx.fillRect(0, 0, preview.width, preview.height);
  for (let y = 0; y < 8; y++) {
    for (let x = 0; x < 8; x++) {
      if (matrix[y][x] === 1) {
        previewCtx.fillStyle = 'black';
        previewCtx.fillRect(x * scale, y * scale, scale, scale);
      }
    }
  }

  // 把 matrix 傳回 Python
  google.colab.kernel.invokeFunction('notebook.predict_digit', [matrix], {});
}
</script>
"""

display(HTML(html_code))

# 3. 在 Python 定義 callback，接收 matrix 並做模型推理
def predict_digit(matrix):
    arr = np.array(matrix).reshape(1, 8, 8).astype("float32")
    prob = model.predict(arr, verbose=0)[0][0]

    if prob >= 0.6:
        result = f"預測機率: {prob:.4f} -> 判斷: 1"
    elif prob <0.6 and prob >=0.5:
        result = f"預測機率: {prob:.4f} -> 判斷: 可能是1"
    elif prob <0.5 and prob >=0.4:
        result = f"預測機率: {prob:.4f} -> 判斷: 可能是3"
    else:
        result = f"預測機率: {prob:.4f} -> 判斷: 3"

    display(HTML(f"<script>document.getElementById('predictOutput').innerText = '{result}';</script>"))

from google.colab import output as colab_output
colab_output.register_callback('notebook.predict_digit', predict_digit)


Saving model.h5 to model (1).h5
✅ 已上傳模型: model (1).h5
✅ 模型加載成功！


Output()